## 카카오 로컬 API 활용
- 도로명 주소 ---> 위도, 경도 좌표

In [1]:
import requests, json
import pandas as pd
from urllib.parse import quote

- API Key

In [2]:
with open('keys/카카오apikey.txt') as file:
    kakao_key = file.read()

- URL 만들기

In [3]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울특별시 영등포구 경인로 지하843 (영등포동3가)'
url = f'{base_url}?query={quote(addr)}'

In [5]:
# Header: Authorization: KakaoAK ${REST_API_KEY}
header = {'Authorization': f'KakaoAK {kakao_key}'}

- Kakao Local API 호출하여 결과 가져오기

In [7]:
result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '서울 영등포구 영등포동3가 33',
    'b_code': '1156010400',
    'h_code': '1156053500',
    'main_address_no': '33',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '영등포구',
    'region_3depth_h_name': '영등포동',
    'region_3depth_name': '영등포동3가',
    'sub_address_no': '',
    'x': '126.906658750671',
    'y': '37.5164625580995'},
   'address_name': '서울 영등포구 경인로 지하 843',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 영등포구 경인로 지하 843',
    'building_name': '영등포역지하도상가',
    'main_building_no': '843',
    'region_1depth_name': '서울',
    'region_2depth_name': '영등포구',
    'region_3depth_name': '영등포동3가',
    'road_name': '경인로',
    'sub_building_no': '',
    'underground_yn': 'Y',
    'x': '126.906658750671',
    'y': '37.5164625580995',
    'zone_no': '07305'},
   'x': '126.906658750671',
   'y': '37.5164625580995'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [8]:
result['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

In [9]:
lat = float(result['documents'][0]['y'])
lng = float(result['documents'][0]['x'])
lat, lng

(37.5164625580995, 126.906658750671)

- 영등포소재 지하철역 정보 완성하기

In [5]:
df = pd.read_csv('data/station_list2.csv')
df

,지역명,측정소명,주소,운영기관,설치년도
0,서울,강남구,서울 강남구 학동로 426 강남구청 별관 1동,서울특별시보건환경연구원,1978.0
1,서울,강동구,서울 강동구 구천면로 42길 59 천호1동 주민센터,서울특별시보건환경연구원,1997.0
2,서울,강북구,서울 강북구 삼양로 139길 49 우이동 주민센터,서울특별시보건환경연구원,1997.0
3,서울,강서구,서울 강서구 강서로 45 다길 71 화곡3동 푸른들청소년도서관,서울특별시보건환경연구원,1992.0
4,서울,관악구,서울 관악구 신림동길 14 신림동 주민센터,서울특별시보건환경연구원,1979.0
...,...,...,...,...,...
520,제주,연동,제주 제주시 연동 322-1 번지 제주특별자치도청 제2청사 제주특별자치도청 제2청사 옥상,제주특별자치도보건환경연구원,2006.0
521,제주,이도동,제주 제주시 이도2동 1176-1(제주시청) 제주시청,제주특별자치도보건환경연구원,1994.0
522,제주,조천읍,제주특별자치도 제주시 조천읍 조천18길 11-1 조천읍 체육관,제주특별자치도보건환경연구원,2020.0
523,제주,한림읍,제주특별자치도 제주시 한림읍 한림중앙로 71-9 한림읍 체육관,제주특별자치도보건환경연구원,2020.0


In [12]:
import folium
import pandas as pd

lat_list, lng_list = [], []

map = folium.Map(location=[df.위도.mean(), df.경도.mean()], zoom_start=13) #Center position
for i in df.index:
    folium.Marker(
        location=[df.위도[i],df.경도[i]],    # WGS84 좌표계: 위도(latitude), 경도(longitude)
        tooltip=df.이름[i],
        popup=folium.Popup(df.주소[i], max_width=200)
    ).add_to(map)

title_html = '<h3 align="center" style="font-size:20px">영등포소재 주요 지하철역</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map

AttributeError: 'DataFrame' object has no attribute '위도'

In [10]:
result

{'documents': [],
 'meta': {'is_end': True, 'pageable_count': 0, 'total_count': 0}}

In [12]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,이름,주소,위도,경도
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가),37.516463,126.906659
1,신도림역,서울특별시 구로구 경인로 625 (신도림동),37.507084,126.884719
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가),37.534957,126.902731
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동),37.521842,126.924485
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가),37.525700,126.896627


In [14]:
df.to_csv('data/영등포소재 지하철역2.csv', index=False)
pd.read_csv('data/영등포소재 지하철역2.csv')

,이름,주소,위도,경도
0,영등포역,서울특별시 영등포구 경인로 지하843 (영등포동3가),37.516463,126.906659
1,신도림역,서울특별시 구로구 경인로 625 (신도림동),37.507084,126.884719
2,당산역,서울특별시 영등포구 당산로 229 (당산동6가),37.534957,126.902731
3,여의도역,서울특별시 영등포구 여의나루로 지하40 (여의도동),37.521842,126.924485
4,영등포구청역,서울특별시 영등포구 당산로 지하121 (당산동3가),37.525700,126.896627
